In [1]:
import os

In [2]:
import random

In [3]:
from os.path import join

In [4]:
data_prefix = "/project/projectdirs/dasrepo/gordon_bell/deep_learning/data/climate/CAM5_0.25"

In [5]:
source_file_prefix = "/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/smaller_dataset"

In [6]:
names = ["2xCO2", "SST_plus2",  "climo"]

In [15]:
name_to_year_dict = dict(zip(names,["0004","0003", "1980"]))

In [19]:
def get_num_tr_val_test(num_ex):
    num_tr = int(0.6*num_ex)
    num_val = int(0.2*num_ex)
    num_test = int(0.2*num_ex)
    return num_tr, num_val, num_test
    

In [20]:
def get_data(dirname, typ="images",seed=11):
    path = join(data_prefix,dirname,typ)
    all_files = os.listdir(path)
    year = name_to_year_dict[dirname]
    files = [join(path,file_) for file_ in all_files if year in file_]
    sort_and_shuffle_list(files,seed=11, num_shuffles=10)
    return files
    

In [21]:
def sort_and_shuffle_list(list_, seed=3, num_shuffles=10):
    list_.sort()
    random.seed(seed)
    for _ in range(num_shuffles):
        random.shuffle(list_)
    return list_
    

In [32]:
def write_train_val_test(datafiles, txt_file, sourcedir):
    num_ex = len(datafiles)
    num_tr, num_val, num_test = get_num_tr_val_test(num_ex)
    
    fname = os.path.basename(txt_file)
    dirname = os.path.dirname(txt_file)
    
    def write_to_file(typ, slice_):
        file_path = join(sourcedir, typ+ "_" + txt_file)
        with open(file_path,"a") as f:
            for datafile in datafiles[slice_]:
                f.writelines(datafile + "\n")
    start = 0
    stop = 0
    for typ,num in [("tr",num_tr), ("val",num_val), ("test", num_test)]:
        stop += num
        slice_ = slice(start,stop)
        fpath = write_to_file(typ, slice_)
        start += stop
    

In [ ]:
if __name__ == "__main__":
    num_crops = 1
    im_listname = "image_files.txt"
    lbl_listname = "label_files.txt"
    crop_listname = "crop_indices.txt"
    im_txt_file = join(source_file_prefix,im_listname)
    lbl_txt_file = join(source_file_prefix,lbl_listname)
    crop_ind_txt_file = join(source_file_prefix,crop_listname)

    for dirname in names:
        im_files, lbl_files =  get_data(dirname,"images"), get_data(dirname,"labels")


        for crop_ind in range(num_crops):
            write_train_val_test(im_files, im_listname, source_file_prefix)
            write_train_val_test(lbl_files, lbl_listname, source_file_prefix)
            for pref in ["tr_", "val_", "test_"]:
                with open(join(source_file_prefix,pref + crop_listname), "a") as f:
                    for _ in lbl_files:
                        f.write(str(crop_ind) + "\n")

In [34]:
    #shuffle everything the same
    for txt_file in os.listdir(source_file_prefix):
        with open(join(source_file_prefix,txt_file), "r") as f:
            random.seed(3)
            lines = f.readlines()
            random.shuffle(lines)
        with open(join(source_file_prefix,txt_file), "w") as f:
            f.writelines(lines)
            
    